This is a codealong to https://keras.io/examples/nlp/lstm_seq2seq/ using my local Anaconda setup

Setup

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [35]:
np.__version__, tf.__version__, keras.__version__

('1.18.5', '2.0.0', '2.2.4-tf')

Download the data

In [36]:
# !!curl -O http://www.manythings.org/anki/fra-eng.zip
# !!unzip fra-eng.zip

Configuration

In [37]:
batch_size = 64  # Batch size for training
epochs = 100  # Number of epochs to train for
latent_dim = 256  # Latent dimensionality of the encoding space
num_samples = 10000  # Number of samples to train on
# Path to the data txt file on disk
data_path = "fra.txt"

Prepare the data

In [38]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

In [39]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [40]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [41]:
print(input_characters)

[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']


In [42]:
print(target_characters)

['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


In [43]:
for i in range(len(input_texts)):
    if len(target_texts[i]) == 59:
        print(input_texts[i], target_texts[i])

I got carded. 	On m'a demandé ma carte d'identité pour vérifier mon âge.



In [44]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [45]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Build the model

In [46]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [47]:
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [48]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

In [49]:
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [50]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [57]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 335872      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  358400      input_4[0][0]                    
                                                                 lstm_2[0][1]               

Train the model

In [52]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [53]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 80s 10ms/sample - loss: 1.3672 - accuracy: 0.9752 - val_loss: 1.3926 - val_accuracy: 0.9831
Epoch 2/100
8000/8000 [==============================] - 101s 13ms/sample - loss: 1.1401 - accuracy: 0.9850 - val_loss: 1.1982 - val_accuracy: 0.9910
Epoch 3/100
8000/8000 [==============================] - 102s 13ms/sample - loss: 1.0142 - accuracy: 0.9896 - val_loss: 1.1021 - val_accuracy: 0.9949
Epoch 4/100
8000/8000 [==============================] - 98s 12ms/sample - loss: 0.9432 - accuracy: 0.9859 - val_loss: 1.0570 - val_accuracy: 0.9607
Epoch 5/100
8000/8000 [==============================] - 101s 13ms/sample - loss: 0.9035 - accuracy: 0.9790 - val_loss: 1.0175 - val_accuracy: 0.9879
Epoch 6/100
8000/8000 [==============================] - 101s 13ms/sample - loss: 0.8736 - accuracy: 0.9754 - val_loss: 0.9915 - val_accuracy: 0.9683
Epoch 7/100
8000/8000 [==============================]

8000/8000 [==============================] - 102s 13ms/sample - loss: 0.5253 - accuracy: 0.9079 - val_loss: 0.8900 - val_accuracy: 0.9247
Epoch 56/100
8000/8000 [==============================] - 103s 13ms/sample - loss: 0.5222 - accuracy: 0.9075 - val_loss: 0.8830 - val_accuracy: 0.9145
Epoch 57/100
8000/8000 [==============================] - 104s 13ms/sample - loss: 0.5198 - accuracy: 0.9067 - val_loss: 0.8880 - val_accuracy: 0.9128
Epoch 58/100
8000/8000 [==============================] - 102s 13ms/sample - loss: 0.5172 - accuracy: 0.9063 - val_loss: 0.8948 - val_accuracy: 0.9252
Epoch 59/100
8000/8000 [==============================] - 102s 13ms/sample - loss: 0.5150 - accuracy: 0.9057 - val_loss: 0.8896 - val_accuracy: 0.9196
Epoch 60/100
8000/8000 [==============================] - 106s 13ms/sample - loss: 0.5126 - accuracy: 0.9054 - val_loss: 0.9006 - val_accuracy: 0.9140
Epoch 61/100
8000/8000 [==============================] - 102s 13ms/sample - loss: 0.5101 - accuracy: 0.904

In [54]:
# Save model
model.save("s2s_100epochs.h5")  # to use TF 2.0, must end with ".h5"

In [60]:
model = keras.models.load_model("s2s_100epochs.h5")

In [61]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 335872      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  358400      input_4[0][0]                    
                                                                 lstm_2[0][1]               

In [63]:
encoder_inputs = model.input[0]  # input_3
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_4
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_5")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_6")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [64]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [65]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [66]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence:          a f    c           !                               
-
Input sentence: Go.
Decoded sentence:          a f    c           !                               
-
Input sentence: Go.
Decoded sentence:          a f    c           !                               
-
Input sentence: Hi.
Decoded sentence:    u    l                                                   
-
Input sentence: Hi.
Decoded sentence:    u    l                                                   
-
Input sentence: Run!
Decoded sentence:                                                             
-
Input sentence: Run!
Decoded sentence:                                                             
-
Input sentence: Run!
Decoded sentence:                                                             
-
Input sentence: Run!
Decoded sentence:                                                             
-
Input sentence: Run!
Decoded sentence:                                               